## Muat <i>libraries</i>

In [2]:
# UNTUK PRAPROSES CITRA
from numpy import expand_dims
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

# UNTUK MEMUAT PRE-TRAINED MODEL & OPTIMIZER
from keras.models import Model
from keras.optimizers import Adam
from keras.applications import MobileNet

# UNTUK JARINGAN SARAF KONVOLUSI
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten

# UNTUK MENDAPATKAN NILAI LOSS PADA SAAT TRAINING
from keras.metrics import categorical_crossentropy

## Muat MobileNet dan buat <i>layers</i> tambahan

In [4]:
# MUAT PRE-TRAINED MODEL(MobileNet) YANG TELAH DILATIH PADA DATASET ImageNet
# include_top = Falses MEMILIKI ARTI BAHWA OUTPUT LAYER YANG BERISI 1000 NODE TIDAK IKUT DIMUAT
base_model = MobileNet(weights = 'imagenet', include_top = False)

# KITA TAMBAHKAN 3 HIIDEN LAYER BARU SEBELUM OUTPUT LAYER
# HANYA BOBOT PADA 3 HIDDEN LAYER INILAH YANG DILATIH
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x) # 1024 NODE DENGAN FUNGSI AKTIVASI ReLU
x = Dense(1024, activation = 'relu')(x) 
x = Dense(512, activation = 'relu')(x)  # 512 NODE DENGAN FUNGSI AKTIVASI ReLU

# OUTPUT LAYER YANG CUMA PUNYA DUA NODE
# BUAT MENGKLASIFIKASIKAN KOCHENG DAN ANJING
preds = Dense(2, activation = 'softmax')(x) # FUNGSI AKTIVASINYA SOFTMAX

## Sesuaikan <i>input layer</i> dan <i>hidden layer</i>'nya

In [5]:
model = Model(inputs=base_model.input, outputs=preds)

## Tentukan <i>hidden</i> layers mana yang bobot'nya tidak dilatih ulang

In [6]:
# SEMUA HIDDEN LAYERS TIDAK DILATIH ULANG
for layer in model.layers:
    layer.trainable = False
    
# DUA PULUH HIDDEN LAYER TERKAHIR AKAN TERLATIH KEMBALI
for layer in model.layers[20:]:
    layer.trainable = True

## Praproses data citra digital'nya

In [8]:
# DATA MASUKAN PADA INPUT LAYER DIPRAPROSES
train_datagen = image.ImageDataGenerator(preprocessing_function = preprocess_input)

# JADIKAN SELURUH DATASET MENJADI DATA TRANINIG :V AWOKWKWKWKWKWKW
train_generator = train_datagen.flow_from_directory('koceng',                 # NAMA FOLDER DATASET
                                                   target_size = (224,224),    # UKURAN CITRA YANG BAKAL DILATIH
                                                   color_mode = 'rgb',         # RED-GREEN-BLUE
                                                   batch_size = 32,            # BATCH UNTUK UPDATE BOBOT & BIAS
                                                   class_mode = 'categorical', # INI UNTUK KLASIFIKASI
                                                   shuffle = True)             # DATA DIAMBIL SECARA ACAK

Found 25000 images belonging to 2 classes.


## Latih ulang MobileNet pada <i>dataset</i> kucing dan anjing

In [7]:
# PARAMETER YANG DIGUNAKAN
model.compile(optimizer = 'Adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size

# LATIH JARINGAN'NYA
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10)

Epoch 1/10
781/781 [==============================] - 637s 816ms/step - loss: 0.1225 - acc: 0.9551
Epoch 2/10
781/781 [==============================] - 628s 804ms/step - loss: 0.0741 - acc: 0.9731
Epoch 3/10
781/781 [==============================] - 1165s 1s/step - loss: 0.0646 - acc: 0.9773
Epoch 4/10
781/781 [==============================] - 785s 1s/step - loss: 0.0518 - acc: 0.9816
Epoch 5/10
781/781 [==============================] - 641s 821ms/step - loss: 0.0486 - acc: 0.9815
Epoch 6/10
781/781 [==============================] - 646s 828ms/step - loss: 0.0400 - acc: 0.9858
Epoch 7/10
781/781 [==============================] - 657s 841ms/step - loss: 0.0382 - acc: 0.9863
Epoch 8/10
781/781 [==============================] - 662s 848ms/step - loss: 0.0376 - acc: 0.9864
Epoch 9/10
781/781 [==============================] - 663s 848ms/step - loss: 0.0392 - acc: 0.9876
Epoch 10/10
781/781 [==============================] - 656s 840ms/step - loss: 0.0240 - acc: 0.9919


Proses pelatihan 25000 gambar dilakukan saat ba'da mahgrib  sampai dengan jam delapan malam di lingkungan fakultas ilmu komputer upnvjatim. Malam-malam digigit nyamuk karena tidak di dalam ruangan(lab) :v

## Uji model

In [13]:
def load_image(img_path):
    # PRAPROSES DATA UJI
    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)
    img_tensor = expand_dims(img_tensor, axis=0)
    img_tensor /= 255.

    return img_tensor

# AMBIL DATA UJI
img_path = 'koceng/dog/dog.18.jpg'
new_image = load_image(img_path)

# PREDIKSI DATA UJI
pred = model.predict(new_image)
# CETAK HASIL PREDIKSI
pred

array([[0.00498788, 0.9950121 ]], dtype=float32)

Probabilitas masukan berupa anjing :

In [14]:
print(pred[0][0])
print(pred[0][1])

0.004987885
0.9950121


Ketika gambar yang dimasukan adalah gambar kucing :

In [10]:
print(pred[0][0])
print(pred[0][1])

1.0
6.1037415e-09


## Simpan model yang telah dilatih

In [8]:
model.save("koceng_anjeng.h5")

Jika diperhatikan dengan seksama urutan eksekusi pada <i>cell</i> di Jupyter Notebook ini ada yang tidak berurutan<br>
Ini dikarenakan Jupyter Notebook yang kami gunakan untuk melatih model CNN telah direfaktorisasi menjadi Jupyter Notebook yang seperti kalian lihat sekarang hehehe